In [69]:
import cv2
import numpy as np
import tensorflow as tf

In [70]:
print(tf.__version__)

2.2.0


In [72]:
large = cv2.imread('hwr-form-6.jpg')
print(large.shape[0],large.shape[1])
h = large.shape[0]
w = large.shape[1]
rgb = cv2.resize(large, (725, 660))
small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
img = large
img2 = small
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
kernel = np.ones((1, 9), np.uint8)
grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)
_, bw = cv2.threshold(grad, 0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 1))
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (12,1))
#connected = cv2.dilate(connected,kernel2, iterations = 1)

cv2.imwrite("/Users/nishith/cv/Lib/site-packages/i1.jpg",connected)
# using RETR_EXTERNAL instead of RETR_CCOMP
contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

mask = np.zeros(bw.shape, dtype=np.uint8)
count=0

text = ""
TEXT = set()
for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    mask[y:y+h, x:x+w] = 0
    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
    
    if r > 0.4 and w > 8 and h > 8 and w>2*h:
        #cv2.rectangle(rgb, (x-1, y-1), (x+w-1, y+h-1), (255, 255, 255), -1)
        m = np.mean(rgb[:, :]) + 5
        TEXT.add((x,y,w,h))
        #cv2.imwrite("words/"+str(idx)+".jpg", rgb[y-10:y+h+5, x-5:x+w+5])
        for j in range(x-5, x+w+5):
            for i in range(y-10, y+h+5):
                rgb[i,j] = m

cv2.imwrite("/Users/nishith/cv/Lib/site-packages/mod_img.jpg",rgb)
cv2.imshow('rects', rgb)
cv2.waitKey(0)

print(TEXT)

2121 1911
{(471, 53, 95, 26), (192, 440, 59, 25), (197, 482, 57, 25), (363, 54, 93, 24), (103, 135, 84, 24), (153, 415, 124, 28)}


In [63]:
img = cv2.imread("mod_img.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grey = cv2.GaussianBlur(grey,(21,21), 0)
th = cv2.adaptiveThreshold(grey,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,33,2)
kernel = np.ones((1,5),np.uint8)

th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result

cv2.imwrite("inter1.jpg", result)

True

In [64]:
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()


for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        
        
for i in boundRect:
    if i[2]<=10 or i[3]<=10:
        boundRect.remove(i)
    if i[2]==img3.shape[1] and i[3]==img3.shape[0]:
        boundRect.remove(i)
        
i=0
j=0
while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1

temp = list()        #stores Radio buttons ,Check boxes and No.s  
buttons = list()     #stores Buttons
images = list()

for i in TEXT.copy():
    x1,y1,w1,h1 = i[:]
    for j in boundRect:
        x,y,w,h = j[:]
        if y1>=y and x<=x1 and y+h>=y1+h1 and x1+w1<=x+w:
            buttons.append(j)
            boundRect.remove(j)
            TEXT.remove(i)
            #cv2.rectangle(img3, (j[0], j[1]), (j[0]+j[2], j[1]+j[3]),(0,0,255), 2)
            break
    
    
for i in TEXT:
    c1 =  i[1]+i[3]/2
    for j in boundRect:
        c = j[1]+j[3]/2
        if abs(c1-c)<=10 and i[0]>j[0]:
            temp.append(j)
            boundRect.remove(j)
            images.append(img3[j[1]-10: j[1]+j[3]+10, j[0]-10: j[0]+j[2]+10])
            #cv2.rectangle(img3, (j[0], j[1]), (j[0]+j[2], j[1]+j[3]),(0,0,255), 2)
            break

        
cv2.imshow("IMAGE", img3)
cv2.waitKey(0)

-1

In [65]:
from tensorflow import keras
model = keras.models.load_model("4mbCAPS.model")

In [66]:
for ig in images:
    cv2.imshow("IMAGE", ig)
    cv2.waitKey()

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)       

In [67]:
mapping = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
p = 0
k = 0
radio_buttons = list()
check_boxes = list()
numbers = list()
for ig1 in images:
    ig1 = cv2.cvtColor(ig1, cv2.COLOR_BGR2GRAY)
    ret2,char = cv2.threshold(ig1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    w,h=char.shape
    d=abs(w-h)
    if d>1:
        if w>h:
            new=np.full((w,w), 255, dtype=np.uint8)
            new[0:w,d//2:d//2+h]=char
        else:
            new=np.full((h,h), 255, dtype=np.uint8)
            new[d//2:d//2+w,0:h]=char
        char=new
    char=cv2.resize(char,(28-2*p,28-2*p))
    new=np.full((28,28), 255, dtype=np.uint8)
    new[p:28-p,p:28-p]=char
    char=new
    char=char.reshape(1,28,28,1)
    classes = model.predict(char)
    letter = mapping[np.argmax(classes[0])]
    if ord(letter)<=57 and ord(letter)>=49:
        numbers.append(temp[k])
    elif ord(letter)==ord("Q") or ord(letter)==ord("O") or ord(letter)==ord("0"):
        radio_buttons.append(temp[k])
    else:
        check_boxes.append(temp[k])
    k+=1
    
    #cv2.imshow("IMAGE", ig1)
    #cv2.waitKey()

In [68]:
for j in radio_buttons:
    cv2.rectangle(img3, (j[0],j[1]), (j[0]+j[2],j[1]+j[3]), (255,0,0), 1)
for j in check_boxes:
        cv2.rectangle(img3, (j[0],j[1]), (j[0]+j[2],j[1]+j[3]), (0,255,0), 1)
for j in numbers:
        cv2.rectangle(img3, (j[0],j[1]), (j[0]+j[2],j[1]+j[3]), (0,0,255), 1)
cv2.imshow("I", img3)
cv2.waitKey()

-1